In [1]:
import pandas as pd
import numpy as np
import random
import sklearn 

from sklearn.model_selection import train_test_split
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GaussianMixture
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from statistics import mode
from collections import Counter


df_ratings = pd.read_csv('df_ratings.csv')
df_prepro = pd.read_csv('df_preprocessed.csv')

C:\Apps\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
df_prepro.head(5)

budget     id  popularity      revenue  runtime  vote_average  \
0  30000000    862    1.360779  373554033.0     81.0           7.7   
1  65000000   8844    1.279713  262797249.0    104.0           6.9   
2         0  15602    1.192545          0.0    101.0           6.5   
3  16000000  31357    1.063445   81452156.0    127.0           6.1   
4         0  11862    1.137880   76578911.0    106.0           5.7   

   vote_count  reliability  greatness_scale  
0      5415.0     4.291533         2.473469  
1      2413.0     2.314364         2.241722  
2        92.0     1.047207         2.090061  
3        34.0     1.016373         1.912237  
4       173.0     1.077845         1.912082

In [3]:
df_ratings = df_ratings.drop(['Unnamed: 0'], axis=1)
df_ratings.head(5)

Action  Adventure  Animation  Comedy  Crime  Documentary  Drama  Family  \
0     0.0        0.0        0.0     0.0    0.0          0.0    2.5     0.0   
1     0.0        0.0        0.0     0.0    0.0          0.0    4.0     0.0   
2     0.0        0.0        0.0     0.0    0.0          0.0    3.0     0.0   
3     0.0        0.0        0.0     0.0    0.0          0.0    4.0     0.0   
4     0.0        0.0        0.0     0.0    0.0          0.0    3.0     0.0   

   Fantasy  Fiction  ...  Music  Mystery  Romance  Science   TV  Thriller  \
0      0.0      0.0  ...    0.0      0.0      0.0      0.0  0.0       0.0   
1      0.0      0.0  ...    0.0      0.0      0.0      0.0  0.0       0.0   
2      0.0      0.0  ...    0.0      0.0      0.0      0.0  0.0       0.0   
3      0.0      0.0  ...    0.0      0.0      0.0      0.0  0.0       0.0   
4      0.0      0.0  ...    0.0      0.0      0.0      0.0  0.0       0.0   

   War  Western    id  userId  
0  0.0      0.0  1371       1  
1  0.0      0.0  1371       4  
2  0.0      0.0  1371       7  
3  0.0      0.0  1371      19  
4  0.0      0.0  1371      21  

[5 rows x 24 columns]

In [4]:
#find 50 best movies.
df_prepo = df_prepro.sort_values('greatness_scale', ascending= False)
popular_id = list(np.array(df_prepo['id'][0:25])) #list of 50 best movies (popularity*reliability)

popular_id

[211672,
 297762,
 321612,
 177572,
 339403,
 283995,
 293660,
 19995,
 245891,
 210577,
 680,
 155,
 271110,
 281338,
 131631,
 166426,
 78,
 24428,
 244786,
 550,
 119450,
 268531,
 198663,
 278,
 118340]

In [5]:
#randomly select 5 of those 25 (that simulates input from a quiz :D)

simulated_choice = random.choices(popular_id, k=5)
simulated_choice

[278, 198663, 118340, 268531, 278]

Now I know someone watched and liked those 5 movies:

1) Find out what HE has in common with grupped customers and pass movies they liked to him. CLUSTER CUSTOMERS
2) Find out what those movies have in common and try to cluster them with others. CLUSTER MOVIES

In [6]:
genres_cols = ['Action', 'Adventure', 'Animation', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Fiction',
       'Foreign', 'History', 'Horror', 'Movie', 'Music', 'Mystery', 'Romance',
       'Science', 'TV', 'Thriller', 'War', 'Western']

km = KMeans(n_clusters=8)

predicted = km.fit_predict(df_ratings[genres_cols])
df_ratings['cluster_km'] = predicted

In [7]:
df_ratings.sort_values('userId').head(10)

Action  Adventure  Animation  Comedy  Crime  Documentary  Drama  Family  \
0        0.0        0.0        0.0     0.0    0.0          0.0    2.5     0.0   
182      0.0        0.0        0.0     2.0    0.0          0.0    0.0     0.0   
235      0.0        0.0        0.0     2.5    2.5          0.0    2.5     0.0   
47       0.0        0.0        0.0     0.0    0.0          0.0    1.0     0.0   
140      0.0        0.0        0.0     2.0    0.0          0.0    2.0     0.0   
93       0.0        0.0        0.0     4.0    0.0          0.0    0.0     0.0   
1475     0.0        0.0        0.0     0.0    0.0          0.0    1.0     0.0   
4437     0.0        0.0        0.0     4.0    0.0          0.0    4.0     0.0   
4515     0.0        0.0        0.0     0.0    0.0          0.0    4.0     0.0   
4759     0.0        0.0        0.0     0.0    0.0          0.0    4.0     0.0   

      Fantasy  Fiction  ...  Mystery  Romance  Science   TV  Thriller  War  \
0         0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
182       0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
235       0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
47        0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
140       0.0      0.0  ...      0.0      2.0      0.0  0.0       0.0  0.0   
93        0.0      0.0  ...      0.0      4.0      0.0  0.0       0.0  0.0   
1475      0.0      0.0  ...      1.0      0.0      0.0  0.0       0.0  0.0   
4437      0.0      0.0  ...      0.0      4.0      0.0  0.0       0.0  0.0   
4515      0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
4759      0.0      0.0  ...      0.0      4.0      0.0  0.0       0.0  0.0   

      Western    id  userId  cluster_km  
0         0.0  1371       1           1  
182       0.0  2294       1           7  
235       0.0  2455       1           4  
47        0.0  1405       1           7  
140       0.0  2193       1           4  
93        0.0  2105       1           4  
1475      0.0   223       2           7  
4437      0.0   509       2           2  
4515      0.0   527       2           1  
4759      0.0   537       2           2  

[10 rows x 25 columns]

In [8]:
df_ratings['cluster_km'].describe()
df_ratings['cluster_km'].unique()

array([1, 7, 4, 2, 3, 0, 5, 6], dtype=int64)

In [9]:
dfs = dict(tuple(df_ratings.groupby('userId')))
dfs[1]

Action  Adventure  Animation  Comedy  Crime  Documentary  Drama  Family  \
0       0.0        0.0        0.0     0.0    0.0          0.0    2.5     0.0   
47      0.0        0.0        0.0     0.0    0.0          0.0    1.0     0.0   
93      0.0        0.0        0.0     4.0    0.0          0.0    0.0     0.0   
140     0.0        0.0        0.0     2.0    0.0          0.0    2.0     0.0   
182     0.0        0.0        0.0     2.0    0.0          0.0    0.0     0.0   
235     0.0        0.0        0.0     2.5    2.5          0.0    2.5     0.0   

     Fantasy  Fiction  ...  Mystery  Romance  Science   TV  Thriller  War  \
0        0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
47       0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
93       0.0      0.0  ...      0.0      4.0      0.0  0.0       0.0  0.0   
140      0.0      0.0  ...      0.0      2.0      0.0  0.0       0.0  0.0   
182      0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   
235      0.0      0.0  ...      0.0      0.0      0.0  0.0       0.0  0.0   

     Western    id  userId  cluster_km  
0        0.0  1371       1           1  
47       0.0  1405       1           7  
93       0.0  2105       1           4  
140      0.0  2193       1           4  
182      0.0  2294       1           7  
235      0.0  2455       1           4  

[6 rows x 25 columns]

In [25]:
mody = []
for value in df_ratings['userId'].unique():
    try:
        print(value, mode(dfs[value]['cluster_km']))
        mody = mody.append(mode(dfs[value]['cluster_km']))
    except:
        d = dfs[1]['cluster_km'].to_list()
        d = Counter(d)
        common = d.most_common(1)
        mody = mody.append(random_choice(common))

1 4
4 1


AttributeError: 'NoneType' object has no attribute 'append'

In [23]:
mody = []
for value in df_ratings['userId'].unique():
    try:
        print(value, mode(dfs[value]['cluster_km']))
        mody = mody.append(mode(dfs[value]['cluster_km']))
    except:
        picked_list = dfs[value]['cluster_km'].to_list().most_common(1))
        mody = mody.append(random.choice(picked_list))
        

SyntaxError: invalid syntax (<ipython-input-23-d24be0defc13>, line 7)

In [13]:
test = [1, 1 , 2, 3, 3]
test= Counter(test)
type(test.most_common(1))

list

In [24]:
d = dfs[1]['cluster_km'].to_list()
d = Counter(d)
common = d.most_common(1)
mody = mody.append(random_choice(common))

AttributeError: 'list' object has no attribute 'most_common'

In [ ]:
mody = []
for value in df_ratings['userId'].unique():
    try:
        print(value, mode(dfs[value]['cluster_km']))
        mody = mody.append(mode(dfs[value]['cluster_km']))
    except:
        picked_list = list(dfs[value]['cluster_km'].most_common(1))
        mody = mody.append(random.choice(picked_list))

In [ ]:
mody